In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import polars as pl
import random
import os
from sklearn.neighbors import KDTree

In [2]:
weapons = ["bomb", "rifle", "arrow"]
arr_random_input = np.random.default_rng().uniform(low=-100,high=100,size=[10_000_000,2])
arr_random_query = np.random.default_rng().uniform(low=-100,high=100,size=[1000,2])
df = pd.DataFrame(arr_random_input, columns=["x","y"])
query_df = pd.DataFrame(arr_random_query, columns=["x","y"])
df["weapon"] = "a"
df["weapon"]=df.weapon.apply(lambda x:random.choice(weapons))
df.to_parquet("notebook_results/input.parquet")
query_df.to_parquet("notebook_results/query.parquet")

### Original kdtree with polars (our current best) -- does around 5.2sec for 10Mx1000

In [3]:
query_df_p = pl.read_parquet("notebook_results/query.parquet")
df_p = pl.read_parquet("notebook_results/input.parquet")

In [4]:
%%timeit
tree = spatial.KDTree(df_p[["x","y"]])
out = []
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0], 
                "weapon": df_p[int(tree.query(row)[1])]["weapon"].item()})

5.23 s ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### New tree with polars

In [7]:
tree = KDTree(df_p[["x","y"]],leaf_size=30)
out = []


(array([[0.02298437]]), array([[1387615]], dtype=int64))

In [16]:
tree.query(query_df_p[1])[1][0][0]

4912397

In [ ]:
int(tree.query(query_df_p[0])[0])

C:\Users\klapc6\AppData\Local\Temp\ipykernel_5832\3910450076.py:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  int(tree.query(query_df_p[0])[0])


0

In [17]:
for row in query_df_p.iter_rows():
    out.append({"dist": tree.query(row)[0][0][0][0], 
                "weapon": df_p[int(tree.query(row)[1])[1][0][0]]["weapon"].item()})

ValueError: Expected 2D array, got 1D array instead:
array=[-62.97846782  81.91153311].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.